<a href="https://colab.research.google.com/github/vincenzomanzoni/quandosaremovaccinati/blob/main/01_Quando_Saremo_Vaccinati_Modello_Italia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quando Saremo Vaccinati

Il codice è volutamente non ottimizzato all'estremo per mantenere una buona leggibilità anche per i lettori non esperti in Python. Si è scelto di usare l'inglese per il nome delle variabili e delle costanti, ma di mantenere l'italiano per il nome delle colonne del dataset.

## Import delle librerie

In [1]:
import pandas as pd
import datetime
import json, requests

from dateutil.parser import parse
from babel.dates import format_date, format_datetime, format_time

## Costanti

In [2]:
INPUT_FILE = "https://raw.githubusercontent.com/italia/covid19-opendata-vaccini/master/dati/somministrazioni-vaccini-summary-latest.json"
LAST_UPDATE = "https://raw.githubusercontent.com/italia/covid19-opendata-vaccini/master/dati/last-update-dataset.json"
ITALIAN_POPULATION = 59641488 # da http://dati.istat.it/Index.aspx?QueryId=18460
PERCENTAGE_FOR_HERD_IMMUNITY = 0.70
DOSES_PER_DAY_ITALIAN_TARGET = 500000

## Caricamento dati e pre-processing

In [3]:
df = pd.read_json(INPUT_FILE, orient="table").set_index("data_somministrazione")
df.sample(3)

,area,totale,sesso_maschile,sesso_femminile,categoria_operatori_sanitari_sociosanitari,categoria_personale_non_sanitario,categoria_altro,categoria_ospiti_rsa,categoria_over80,categoria_forze_armate,categoria_personale_scolastico,prima_dose,seconda_dose,codice_NUTS1,codice_NUTS2,codice_regione_ISTAT,nome_area
data_somministrazione,,,,,,,,,,,,,,,,,
2021-03-24,MAR,7036,2982,4054,484,20,961,187,4728,64,592,3367,3669,ITI,ITI3,11,Marche
2021-02-03,PUG,5031,2002,3029,4366,469,9,184,0,0,3,730,4301,ITF,ITF4,16,Puglia
2021-03-26,LIG,9094,3843,5251,602,182,2751,65,3881,316,1297,5940,3154,ITC,ITC3,7,Liguria


In [4]:
last_update_json = json.loads(requests.get(LAST_UPDATE).text)
last_update = parse(last_update_json["ultimo_aggiornamento"])
print("Ultimo aggiornamento:", format_datetime(last_update, format="long", locale="it"))

Ultimo aggiornamento: 18 aprile 2021 18:51:10 UTC


Come si vede, ogni riga del dataset corrisponde a una data e una regione. Il dataset viene raggruppato per data di somministazione per avere il dato nazionale per giorno. Per ogni giorno vengono quindi sommati il numero delle prime dosi (`prima_dose`) e il numero di seconde dosi (`seconda_dose`).

In [5]:
df_italy = df.groupby("data_somministrazione") \
  .sum()[['prima_dose', 'seconda_dose']] \
  .copy()
df_italy.sample(3)

,prima_dose,seconda_dose
data_somministrazione,,
2021-03-13,127097,48242
2021-02-10,44922,27270
2021-01-26,4073,75800


Sul dataset con i dati giornalieri nazionali vengono calcolati, per ogni giorno:

*   `prima_dose_rm7`: media delle prime dosi somministrate negli ultimi 7 giorni.
*   `prima_dose_cs`: somma delle prime dosi somministrate fino a quel giorno.
*   `seconda_dose_rm7`: media delle seconde dosi somministrate negli ultimi 7 giorni.
*   `seconda_dose_cs`: somma delle seconde dosi somministrate fino a quel giorno.

In [6]:
df_italy["prima_dose_rm7"] = df_italy.rolling(7).prima_dose.mean()
df_italy['prima_dose_cs']  = df_italy.prima_dose.cumsum()

df_italy["seconda_dose_rm7"] = df_italy.rolling(7).seconda_dose.mean()
df_italy['seconda_dose_cs']  = df_italy.seconda_dose.cumsum()

In [7]:
df_italy.tail(3)

,prima_dose,seconda_dose,prima_dose_rm7,prima_dose_cs,seconda_dose_rm7,seconda_dose_cs
data_somministrazione,,,,,,
2021-04-16,266692,94304,222305.285714,10333999,71707.428571,4337386
2021-04-17,259385,78726,224371.714286,10593384,73277.000000,4416112
2021-04-18,116813,41967,218559.571429,10710197,73420.142857,4458079


In [8]:
last_value_date = df_italy.tail(1).index[0].to_pydatetime()
if (last_value_date.year == last_update.year) \
    and (last_value_date.month == last_update.month) \
    and (last_value_date.day == last_update.day):
  simulation_day = df_italy.tail(2).head(1)
else:
  simulation_day = df_italy.tail(1)

In [9]:
first_dose_cs = simulation_day['prima_dose_cs'][0]
first_dose_rm7 = simulation_day['prima_dose_rm7'][0]

second_dose_cs = simulation_day['seconda_dose_cs'][0]
second_dose_rm7 = simulation_day['seconda_dose_rm7'][0]

days_to_partial_herd_immunity = round(
      (ITALIAN_POPULATION * PERCENTAGE_FOR_HERD_IMMUNITY - first_dose_cs) 
      / first_dose_rm7
    )
day_of_partial_herd_immunity = simulation_day.index[0] + \
  datetime.timedelta(days=days_to_partial_herd_immunity)

days_to_complete_herd_immunity = round(
        (ITALIAN_POPULATION * PERCENTAGE_FOR_HERD_IMMUNITY - second_dose_cs) 
        / second_dose_rm7
      )
day_of_complete_herd_immunity = simulation_day.index[0] + \
  datetime.timedelta(days=days_to_complete_herd_immunity)

In [10]:
print("Prime dosi   - Giorni al 70%:", days_to_partial_herd_immunity)
print("Prime dosi   - Giorno al 70%:", format_date(day_of_partial_herd_immunity, format="long", locale="it"))
print()
print("Seconde dosi - Giorni al 70%:", days_to_complete_herd_immunity)
print("Seconde dosi - Giorno al 70%:", format_date(day_of_complete_herd_immunity, format="long", locale="it"))
print()
print("Giorno di simulazione:       ", format_date(simulation_day.index[0], format="long", locale="it"))
print("Ultimo aggiornamento:        ", format_datetime(last_update, format="long", locale="it"))

Prime dosi   - Giorni al 70%: 139
Prime dosi   - Giorno al 70%: 3 settembre 2021

Seconde dosi - Giorni al 70%: 509
Seconde dosi - Giorno al 70%: 8 settembre 2022

Giorno di simulazione:        17 aprile 2021
Ultimo aggiornamento:         18 aprile 2021 18:51:10 UTC
